# Argumentation Abstraite et Structuree

Ce notebook couvre les frameworks Dung, ASPIC+, DeLP, ABA et ASP.

**Pre-requis**: Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement.

**Navigation**: [Precedent](Tweety-4-Belief-Revision.ipynb) | [Suivant](Tweety-6-Advanced-Argumentation.ipynb)


In [ ]:
# --- Initialisation JVM Tweety ---
# Cette cellule verifie et demarre la JVM si necessaire.
# Pre-requis: Executez d'abord Tweety-1-Setup.ipynb pour telecharger les JARs.

print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib

# Check if JVM already running
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    # Try to start JVM with existing libs
    LIB_DIR = pathlib.Path("libs")
    if not LIB_DIR.exists():
        LIB_DIR = pathlib.Path("Argument_Analysis/libs")

    if not LIB_DIR.exists():
        print("ERREUR: Dossier 'libs/' non trouve.")
        print("Veuillez d'abord executer Tweety-1-Setup.ipynb pour telecharger les JARs.")
    else:
        jar_list = [str(p.resolve()) for p in LIB_DIR.glob("*.jar")]
        if not jar_list:
            print("ERREUR: Aucun JAR trouve dans libs/.")
        else:
            # Find JAVA_HOME
            java_home = os.getenv("JAVA_HOME")
            jdk_portable = pathlib.Path("jdk-17-portable")
            if not jdk_portable.exists():
                jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")

            if jdk_portable.exists():
                for subdir in jdk_portable.iterdir():
                    if subdir.is_dir() and (subdir / "bin").exists():
                        java_home = str(subdir.resolve())
                        os.environ['JAVA_HOME'] = java_home
                        break

            if java_home:
                classpath = os.pathsep.join(jar_list)
                try:
                    jpype.startJVM("-ea", f"-Djava.class.path={classpath}", convertStrings=False)
                    jpype.imports.registerDomain("org")
                    jpype.imports.registerDomain("java")
                    jpype.imports.registerDomain("net")
                    print(f"JVM demarree avec {len(jar_list)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"ERREUR au demarrage JVM: {e}")
            else:
                print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")

if jvm_ready:
    print("JVM prete pour Tweety.")
else:
    print("JVM non disponible. Certaines cellules echoueront.")



## Partie 4 : Argumentation Abstraite et Structurée
<a id="partie4"></a>

Nous entrons maintenant au cœur de l'argumentation computationnelle, en commençant par le cadre fondateur de Dung et en progressant vers des approches qui prennent en compte la structure interne des arguments. Cette partie est souvent plus stable car elle repose sur des modules centraux de Tweety.

### 4.1 Cadres d'Argumentation Abstraits (Dung)
<a id="4.1"></a>

Le cadre de Dung (Abstract Argumentation Framework - AAF) est la base de nombreuses extensions. Il modélise les conflits entre arguments de manière abstraite.

* Un ensemble d'**Arguments** (`Argument`) considérés comme des entités atomiques.
* Une relation d'**Attaque** (`Attack`) binaire entre arguments ($a$ attaque $b$).
* L'ensemble forme une **Théorie de Dung** (`DungTheory`).

L'objectif est de déterminer des ensembles d'arguments collectivement acceptables, appelés **extensions**, selon différentes **sémantiques** (Conflict-Free, Admissible, Complete, Grounded, Preferred, Stable, etc.). Tweety fournit des **raisonneurs** (`org.tweetyproject.arg.dung.reasoner.*`) pour calculer ces extensions.

* **Conflict-Free (CF)**: Aucun argument dans l'extension n'attaque un autre argument de l'extension.
* **Admissible (ADM)**: Conflict-Free + chaque argument de l'extension est défendu par l'extension contre ses attaquants.
* **Complete (COM)**: Admissible + contient tous les arguments qu'elle défend.
* **Grounded (GR)**: L'extension complète minimale (calculée itérativement, sceptique).
* **Preferred (PRF)**: Une extension complète maximale (par inclusion). Il peut y en avoir plusieurs (crédule).
* **Stable (ST)**: Conflict-Free + attaque tous les arguments qui ne sont pas dans l'extension. Peut ne pas exister, mais si elle existe, elle est aussi Preferred et Complete.
* **CF2**: Sémantique gérant les cycles impairs via la décomposition en Composantes Fortement Connexes (SCC).

Tweety fournit des **raisonneurs** (`org.tweetyproject.arg.dung.reasoner.*`) pour calculer ces extensions (ex: `SimpleGroundedReasoner`, `SimpleStableReasoner`, `SccCF2Reasoner`).

In [ ]:
# --- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
print("\n--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple Dung...")
    try:
        # Imports nécessaires pour Dung
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        # Importer plusieurs raisonneurs
        from org.tweetyproject.arg.dung.reasoner import (
            SimpleGroundedReasoner, SimpleStableReasoner, SimplePreferredReasoner,
            SimpleCompleteReasoner, SimpleAdmissibleReasoner, SimpleConflictFreeReasoner
        )
        # Importer Semantics si besoin pour certains raisonneurs plus avancés (non utilisés ici)
        # from org.tweetyproject.arg.dung.semantics import Semantics
        from java.util import Collection # Pour vérifier taille retour

        print("✔️ Imports Dung réussis.")

        # --- Exemple 1: A <-> B -> C ---
        af1 = DungTheory()
        a1 = Argument("a1"); b1 = Argument("b1"); c1 = Argument("c1")
        # Ajouter des arguments
        af1.add(a1); af1.add(b1); af1.add(c1)
        # Ajouter des attaques
        af1.add(Attack(a1, b1)); af1.add(Attack(b1, a1)); af1.add(Attack(b1, c1))

        print("\n--- AF1: a1 <-> b1 -> c1 ---")
        print("Cadre :", af1)

        # Calculer les extensions pour différentes sémantiques
        print("\nCalcul des extensions pour AF1:")
        try:
            # Grounded: getModel() retourne une Extension, getModels() retourne une Collection<Extension>
            grounded_ext = SimpleGroundedReasoner().getModel(af1) # Plus direct pour grounded
            print(f" - Grounded   : {{{grounded_ext}}}") # Afficher comme un ensemble pour la cohérence
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts = SimpleStableReasoner().getModels(af1)
            print(f" - Stable     ({stable_exts.size()}):", stable_exts)
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts = SimplePreferredReasoner().getModels(af1)
            print(f" - Preferred  ({preferred_exts.size()}):", preferred_exts)
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts = SimpleCompleteReasoner().getModels(af1)
            print(f" - Complete   ({complete_exts.size()}):", complete_exts)
        except Exception as e: print(f"   Erreur Complete: {e}")
        try:
            admissible_exts = SimpleAdmissibleReasoner().getModels(af1)
            # Convertir la Collection Java en liste Python pour len()
            # Note: Cela peut être coûteux si la collection est énorme
            admissible_list = list(admissible_exts)
            print(f" - Admissible ({len(admissible_list)}):", admissible_exts) # Afficher la collection Java
        except Exception as e: print(f"   Erreur Admissible: {e}")
        try:
            conflict_free_exts = SimpleConflictFreeReasoner().getModels(af1)
            cf_list = list(conflict_free_exts)
            print(f" - ConflictFree ({len(cf_list)}):", conflict_free_exts) # Attention, peut être très grand
        except Exception as e: print(f"   Erreur ConflictFree: {e}")


        # --- Exemple 2: Cycle a->b->c->a ---
        af_cycle = DungTheory()
        a_cy = Argument("a_cy"); b_cy = Argument("b_cy"); c_cy = Argument("c_cy")
        af_cycle.add(a_cy); af_cycle.add(b_cy); af_cycle.add(c_cy)
        af_cycle.add(Attack(a_cy, b_cy)); af_cycle.add(Attack(b_cy, c_cy)); af_cycle.add(Attack(c_cy, a_cy))

        print("\n--- AF Cycle: a -> b -> c -> a ---")
        print("Cadre :", af_cycle)
        print("\nCalcul des extensions pour AF Cycle:")
        try:
            print(" - Grounded   :", SimpleGroundedReasoner().getModel(af_cycle)) # {}
        except Exception as e: print(f"   Erreur Grounded: {e}")
        try:
            stable_exts_cy = SimpleStableReasoner().getModels(af_cycle)
            print(f" - Stable     ({stable_exts_cy.size()}):", stable_exts_cy) # {}
        except Exception as e: print(f"   Erreur Stable: {e}")
        try:
            preferred_exts_cy = SimplePreferredReasoner().getModels(af_cycle)
            print(f" - Preferred  ({preferred_exts_cy.size()}):", preferred_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Preferred: {e}")
        try:
            complete_exts_cy = SimpleCompleteReasoner().getModels(af_cycle)
            print(f" - Complete   ({complete_exts_cy.size()}):", complete_exts_cy) # [{}]
        except Exception as e: print(f"   Erreur Complete: {e}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour l'Argumentation Abstraite : {e}")
        print("   Vérifiez le JAR 'org.tweetyproject.arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple Dung: {e_java.message()}")
        # print(e_java.stacktrace()) # Décommenter pour trace Java complète
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.1 Cadres d'Argumentation Abstraits (Dung) : Bases et Sémantiques ---
❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.


#### 4.1.2 Sémantique CF2
<a id="4.1.2"></a>

La sémantique CF2 ([Baroni, Giacomin, Guida, *SCC-recursive semantics for abstract argumentation frameworks*, 2005](https://link.springer.com/chapter/10.1007/11557791_3)) est une alternative aux sémantiques classiques qui gère différemment les cycles, notamment les cycles impairs. Elle se base sur une décomposition du graphe d'argumentation en Composantes Fortement Connexes (SCCs). Tweety fournit le `SccCF2Reasoner`.

In [ ]:
# --- 4.1.2 Sémantique CF2 ---
print("\n--- 4.1.2 Sémantique CF2 ---")

# Vérifier si la JVM est prête
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple CF2...")
    try:
        # Imports (peuvent être déjà faits, mais sécurité)
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.reasoner import SccCF2Reasoner
        from org.tweetyproject.arg.dung.semantics import Extension # Pour type hinting éventuel
        from java.util import Collection

        print("✔️ Imports Dung (pour CF2) réussis.")

        # --- Exemple AF2 : Cycle a->b->c->d->e->a, e->f ---
        # (Recréation pour autonomie de la cellule)
        af2 = DungTheory()
        args_cf2_map = {name: Argument(name) for name in "abcdef"} # Utiliser un dictionnaire
        for arg in args_cf2_map.values(): af2.add(arg)

        attacks_cf2 = [("a","b"), ("b","c"), ("c","d"), ("d","e"), ("e","a"), ("e","f")]
        for s, t in attacks_cf2:
             # S'assurer que les arguments existent avant d'ajouter l'attaque
             if s in args_cf2_map and t in args_cf2_map:
                  af2.add(Attack(args_cf2_map[s], args_cf2_map[t]))
             else:
                  print(f"WARN: Argument(s) non trouvé(s) pour l'attaque ({s},{t})")

        print("\n--- AF pour CF2 : Cycle a->b->c->d->e->a, e->f ---")
        print("Cadre :", af2)

        # --- Raisonnement CF2 ---
        cf2_reasoner = SccCF2Reasoner()
        print("\nCalcul des extensions CF2:")
        try:
            cf2_extensions_collection = cf2_reasoner.getModels(af2) # Retourne Collection<Extension>
            if cf2_extensions_collection.isEmpty():
                 print("  (Aucune extension CF2 trouvée)")
            else:
                 # Itérer sur la collection Java
                 ext_iterator = cf2_extensions_collection.iterator()
                 count = 0
                 while ext_iterator.hasNext():
                      ext = ext_iterator.next()
                      # Pour un affichage plus propre des arguments dans l'extension:
                      args_in_ext = ", ".join(sorted([str(arg.getName()) for arg in ext]))
                      print(f"  - {{{args_in_ext}}}")
                      count += 1
                 print(f"  ({count} extension(s) trouvée(s))")

        except jpype.JException as e_cf2_java:
             print(f"  ❌ Erreur Java lors du raisonnement CF2: {e_cf2_java.message()}")
             # print(e_cf2_java.stacktrace())
        except Exception as e_cf2_py:
              print(f"  ❌ Erreur Python lors du raisonnement CF2: {e_cf2_py}")

    # Gestion globale
    except ImportError as e:
        print(f"❌ Erreur d'import pour CF2 : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple CF2: {e_java.message()}")
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple CF2: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.2 Sémantique CF2 ---
❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.


#### 4.1.3 Génération de Cadres d'Argumentation
<a id="4.1.3"></a>

Tweety fournit des outils pour générer aléatoirement des cadres d'argumentation abstraits (`DungTheory`), ce qui est utile pour les tests, benchmarks ou simulations. `DefaultDungTheoryGenerator` est un générateur simple paramétrable.

In [ ]:
# --- 4.1.3 Génération de Cadres Dung ---
print("\n--- 4.1.3 Génération de Cadres Dung ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple de génération...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
        from org.tweetyproject.arg.dung.util import DefaultDungTheoryGenerator, DungTheoryGenerationParameters

        print("✔️ Imports pour génération Dung réussis.")

        # Paramètres de génération
        params = DungTheoryGenerationParameters()
        params.numberOfArguments = 6
        params.attackProbability = 0.25
        # CORRECTION: La ligne suivante est retirée car le champ n'existe probablement plus
        # params.allowSelfAttacks = False
        print(f"ℹ️ Paramètres de génération: {params.numberOfArguments} args, proba_attaque={params.attackProbability} (comportement par défaut pour auto-attaques)")

        generator = DefaultDungTheoryGenerator(params)

        # Générer UN cadre
        generated_af = generator.next()

        print("\nCadre généré aléatoirement :")
        print(f"  Arguments: {generated_af.getNodes()}")
        print(f"  Attaques : {generated_af.getAttacks()}")
        print(f"  Représentation compacte: {generated_af}")

    except ImportError as e:
        print(f"❌ Erreur d'import pour la génération Dung : {e}. Vérifiez le JAR 'arg.dung'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple génération Dung: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple génération Dung: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.1.3 Génération de Cadres Dung ---
❌ ERREUR: JVM non démarrée.


#### 4.1.4 Apprentissage de Cadres d'Argumentation (depuis Labellisations)
<a id="4.1.4"></a>

L'apprentissage de cadre vise à reconstruire un cadre d'argumentation (ou un ensemble de cadres possibles) à partir d'informations partielles, typiquement des exemples de labellisations d'arguments (IN, OUT, UNDEC) selon une certaine sémantique. Tweety propose `SimpleAFLearner` pour cela.

In [ ]:
# --- 4.1.4 Apprentissage de Cadres Dung (depuis Labellisations) ---
print("\n--- 4.1.4 Apprentissage de Cadres Dung ---")

print("!! SECTION COMMENTÉE !!")
print("L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety")
print("(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling")
print("pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.")

# if not jvm_ready:
#     print("❌ ERREUR: JVM non démarrée.")
# else:
#     print("ℹ️ JVM prête. Exécution de l'exemple d'apprentissage...")
#     try:
#         # Imports
#         import jpype
#         from jpype.types import *
#         from org.tweetyproject.arg.dung.syntax import DungTheory, Argument, Attack
#         from org.tweetyproject.arg.dung.learning import SimpleAFLearner
#         from org.tweetyproject.arg.dung.learning.syntax import Entity
#         from org.tweetyproject.arg.dung.semantics import Semantics
#         # Import de Label retiré (correction précédente)
#
#         print("✔️ Imports pour apprentissage Dung réussis.")
#
#         # 1. Définir le cadre "caché"
#         hidden_af = DungTheory()
#         a_learn = Argument("a"); b_learn = Argument("b"); c_learn = Argument("c")
#         hidden_af.add(a_learn); hidden_af.add(b_learn); hidden_af.add(c_learn)
#         hidden_af.add(Attack(a_learn, b_learn)); hidden_af.add(Attack(b_learn, a_learn)); hidden_af.add(Attack(b_learn, c_learn))
#         print("\nCadre caché (à apprendre):", hidden_af)
#
#         # 2. Créer l'Oracle
#         oracle = Entity(hidden_af)
#         arguments_known = oracle.getArguments()
#         print("Arguments connus par l'apprenant:", arguments_known)
#
#         # 3. Créer l'Apprenant
#         learner = SimpleAFLearner(arguments_known)
#         print(f"État initial apprenant: {learner.getNumberOfFrameworks()} cadres compatibles possibles.")
#
#         # 4. Apprendre depuis des labellisations
#         # Labellisation Stable
#         print("\nApprentissage depuis labellisation Stable:")
#         try:
#              labeling_stable = oracle.getLabeling(Semantics.STABLE_SEMANTICS) # C'est ici que l'erreur se produit
#              print(f"  Oracle fournit Labellisation STABLE: {labeling_stable}")
#              learner.learnLabeling(labeling_stable)
#              print(f"  Après STABLE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_stable:
#               print(f"  Erreur lors de l'obtention/apprentissage de la labellisation Stable: {e_stable.message()}")
#               # Afficher la stacktrace Java peut aider à confirmer l'erreur interne
#               # print(e_stable.stacktrace())
#
#         # Labellisation Conflict-Free (pourrait aussi échouer)
#         print("\nApprentissage depuis labellisation Conflict-Free:")
#         try:
#              labeling_cf = oracle.getLabeling(Semantics.CONFLICTFREE_SEMANTICS)
#              print(f"  Oracle fournit Labellisation CONFLICT_FREE: {labeling_cf}")
#              learner.learnLabeling(labeling_cf)
#              print(f"  Après CONFLICT_FREE: {learner.getNumberOfFrameworks()} cadres compatibles restants.")
#         except jpype.JException as e_cf:
#              print(f"  Erreur lors de l'obtention/apprentissage de la labellisation ConflictFree: {e_cf.message()}")
#
#
#         # 5. Obtenir le(s) cadre(s) appris
#         # Ce code ne sera probablement pas atteint
#         learned_af = learner.getModel()
#         print("\nCadre appris (un des cadres compatibles) :")
#         print(str(learned_af))
#
#     # ... (Blocs except ImportError, JException, Exception comme avant) ...
#     except ImportError as e:
#        print(f"❌ Erreur d'import pour l'apprentissage Dung : {e}.")
#     except jpype.JException as e_java:
#        print(f"❌ Erreur Java générale dans l'exemple apprentissage Dung: {e_java.message()}")
#     except Exception as e_gen:
#        print(f"❌ Erreur Python inattendue dans l'exemple apprentissage Dung: {e_gen}")
#        import traceback
#        traceback.print_exc()


--- 4.1.4 Apprentissage de Cadres Dung ---
!! SECTION COMMENTÉE !!
L'exécution de cet exemple échoue en raison d'une ClassCastException interne à Tweety
(Tautology cannot be cast to AssociativePlFormula) lors de l'appel à getLabeling/learnLabeling
pour le cadre d'argumentation spécifique utilisé ici. Ceci semble être un bug interne.


### 4.2 ASPIC+
<a id="4.2"></a>

ASPIC+ est un framework mature et largement utilisé pour l'argumentation **structurée**. Contrairement à Dung où les arguments sont abstraits, ASPIC+ construit des arguments logiques à partir d'une base de connaissances et de règles d'inférence, en distinguant règles strictes et règles défaisables.

* **Base de Connaissances (KB)**: Contient des axiomes (faits certains) et des règles.
* **Règles d'inférence**:
    * Strictes (`StrictInferenceRule`, `->`): Si les prémisses sont acceptées, la conclusion l'est nécessairement.
    * Défaisables (`DefeasibleInferenceRule`, `=>`): Si les prémisses sont acceptées, la conclusion l'est plausiblement, mais la règle elle-même peut être attaquée (undercutting) ou la conclusion réfutée (rebutting).
* **Préférences**: Un ordre (souvent partiel) peut être défini sur les règles défaisables pour résoudre les attaques "rebutting".
* **Arguments**: Construits par chaînage de règles depuis les axiomes.
* **Attaques**: Peuvent viser la conclusion (rebutting), une sous-conclusion (undermining sur conclusion intermédiaire) ou une règle défaisable (undercutting).
* **`AspicArgumentationTheory`**: Représente la théorie ASPIC+. Nécessite un `RuleFormulaGenerator` pour spécifier la logique sous-jacente (ex: `PlFormulaGenerator` pour la logique propositionnelle).
* **Conversion vers Dung (`asDungTheory`)**: Permet d'analyser la théorie ASPIC+ en la transformant en un AAF standard, sur lequel on peut appliquer les sémantiques de Dung (Grounded, Preferred, etc.).

L'exemple suivant utilise la logique propositionnelle comme langage sous-jacent, basé sur `AspicExample.java`.

In [ ]:
# --- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---
print("\n--- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ASPIC+...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        # Logique Propositionnelle comme base
        from org.tweetyproject.logics.pl.syntax import Proposition, Negation, PlFormula
        # Classes ASPIC+
        from org.tweetyproject.arg.aspic.syntax import AspicArgumentationTheory, DefeasibleInferenceRule, StrictInferenceRule
        from org.tweetyproject.arg.aspic.ruleformulagenerator import PlFormulaGenerator # Crucial pour PL
        # Classes Dung pour la conversion et le raisonnement
        from org.tweetyproject.arg.dung.syntax import Argument as DungArgument, Attack as DungAttack, DungTheory
        from org.tweetyproject.arg.dung.reasoner import SimpleGroundedReasoner

        print("✔️ Imports ASPIC+ (PL) et Dung réussis.")

        # --- Théorie ASPIC+ ---
        # Nécessite un générateur pour la logique sous-jacente (ici PL)
        pl_formula_generator = PlFormulaGenerator()
        aspic_theory = AspicArgumentationTheory(pl_formula_generator)
        # Spécifier aussi le générateur pour les formules DANS les règles
        aspic_theory.setRuleFormulaGenerator(pl_formula_generator)

        # Propositions
        a = Proposition("a"); b = Proposition("b"); c = Proposition("c"); d_prop = Proposition("d") # Renommé

        # Règles Défaisables (=>)
        # r1: b, c => a
        r1_def = DefeasibleInferenceRule()
        r1_def.setConclusion(a)
        r1_def.addPremise(b); r1_def.addPremise(c)
        aspic_theory.addRule(r1_def)

        # r2: b => d
        r2_def = DefeasibleInferenceRule()
        r2_def.setConclusion(d_prop)
        r2_def.addPremise(b)
        aspic_theory.addRule(r2_def)

        # r3: a => !d
        r3_def = DefeasibleInferenceRule()
        r3_def.setConclusion(Negation(d_prop))
        r3_def.addPremise(a)
        aspic_theory.addRule(r3_def)

        # (Pas de règles strictes dans cet exemple)

        # Axiomes (faits certains ->)
        aspic_theory.addAxiom(b) # -> b
        aspic_theory.addAxiom(c) # -> c

        print("\n--- Théorie ASPIC+ ---")
        print(str(aspic_theory)) # Utilise toString()

        # --- Conversion en Cadre de Dung ---
        print("\n--- Conversion en AF de Dung ---")
        try:
            # asDungTheory() génère le graphe d'attaque basé sur la théorie ASPIC+
            dung_equivalent = aspic_theory.asDungTheory()

            print("Arguments générés par ASPIC+:")
            args_aspic = dung_equivalent.getNodes()
            if args_aspic.isEmpty():
                print("  (Aucun argument généré)")
            else:
                for arg in args_aspic:
                    # L'affichage d'un argument ASPIC+ peut être complexe
                    print(f"  - {arg}")

            print("\nAttaques générées par ASPIC+:")
            attacks_aspic = dung_equivalent.getAttacks()
            if attacks_aspic.isEmpty():
                 print("  (Aucune attaque générée)")
            else:
                for att in attacks_aspic:
                    print(f"  - {att}")

            # --- Raisonnement sur l'AF équivalent ---
            print("\nRaisonnement sur l'AF de Dung équivalent:")
            grounded_reasoner_dung = SimpleGroundedReasoner()
            grounded_extension = grounded_reasoner_dung.getModel(dung_equivalent)
            print(f" - Extension Grounded : {grounded_extension}")

            # Pourrait-on vérifier si 'a' est acceptable ?
            # Il faudrait trouver l'argument ASPIC correspondant à 'a'
            # C'est plus complexe, on se contente de l'extension pour l'instant.

        except jpype.JException as e_conv_java:
            print(f"❌ Erreur Java lors de la conversion ASPIC->Dung: {e_conv_java.message()}")
            # print(e_conv_java.stacktrace())
        except Exception as e_conv_py:
            print(f"❌ Erreur Python lors de la conversion ASPIC->Dung: {e_conv_py}")


    except ImportError as e:
        print(f"❌ Erreur d'import pour ASPIC+ : {e}. Vérifiez le JAR 'arg.aspic'.")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple ASPIC+: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple ASPIC+: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---
❌ ERREUR: JVM non démarrée.


### 4.3 Defeasible Logic Programming (DeLP)
<a id="4.3"></a>

*(Section à compléter avec l'exemple `DeLPExample.java`)*

DeLP combine la programmation logique avec le raisonnement défaisable. Il utilise des règles strictes et des règles défaisables (`-<`). Un argument est construit pour supporter un littéral, et la notion de "warrant" (justification) est déterminée en comparant les arguments pour et contre ce littéral, en utilisant un critère de comparaison comme la spécificité généralisée.

In [ ]:
# --- 4.3 Defeasible Logic Programming (DeLP) ---
print("\n--- 4.3 Defeasible Logic Programming (DeLP) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple DeLP...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        import pathlib
        from java.io import StringReader
        from java.util import ArrayList

        # Imports DeLP
        from org.tweetyproject.arg.delp.parser import DelpParser
        from org.tweetyproject.arg.delp.reasoner import DelpReasoner
        from org.tweetyproject.arg.delp.semantics import GeneralizedSpecificity
        from org.tweetyproject.arg.delp.syntax import DefeasibleLogicProgram

        # Imports FOL/Commons nécessaires
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature, FolAtom
        # ** CORRECTION: Importer Constant et Predicate depuis commons.syntax **
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate

        print("✔️ Imports DeLP, FOL et Commons nécessaires réussis.")

        # --- Parsing du programme DeLP ---
        # (Code de chargement depuis fichier ou chaîne inchangé)
        delp_filename = "birds2.txt"
        delp_filepath = pathlib.Path("resources") / delp_filename
        delp_program = None
        parser_delp = DelpParser() # Parser pour le programme

        if not delp_filepath.is_file():
            print(f"❌ ERREUR: Fichier DeLP requis '{delp_filepath}' non trouvé !")
            print("   Utilisation d'un exemple intégré (birds.txt simplifié).")
            birds_program_str = """
            Bird(X) <- Chicken(X). Bird(X) <- Penguin(X). ~Flies(X) <- Penguin(X).
            Chicken(tina). Penguin(tweety). Scared(tina).
            Flies(X) -< Bird(X). ~Flies(X) -< Chicken(X). Flies(X) -< Chicken(X), Scared(X).
            Nests_in_trees(X) -< Flies(X).
            """
            try:
                 string_reader = StringReader(birds_program_str)
                 delp_program = parser_delp.parseBeliefBase(string_reader); string_reader.close()
                 print("✔️ Programme chargé avec succès depuis la chaîne intégrée.")
            except Exception as e_str: print(f"❌ Erreur parsing chaîne intégrée: {e_str}"); delp_program = None
        else:
            print(f"\nChargement du programme DeLP depuis: {delp_filepath}")
            try:
                 delp_program = parser_delp.parseBeliefBaseFromFile(str(delp_filepath))
                 print("✔️ Programme chargé avec succès depuis le fichier.")
            except Exception as e_file: print(f"❌ Erreur chargement fichier: {e_file}"); delp_program = None


        # --- Raisonnement DeLP ---
        if delp_program is not None:
            print("\nProgramme DeLP chargé:\n", str(delp_program))

            # DEBUG Signature (inchangé)
            sig_delp = None
            try:
                sig_delp = delp_program.getSignature()
                print("\nDEBUG Signature extraite par DelpParser:")
                print(sig_delp)
                if hasattr(sig_delp, 'getPredicates'): print("Predicates:", sig_delp.getPredicates())
                if hasattr(sig_delp, 'getConstants'): print("Constants:", sig_delp.getConstants())
            except Exception as e_sig: print(f"⚠️ Erreur récupération/affichage signature: {e_sig}")

            # Construction programmatique des requêtes (inchangée, mais utilise les classes importées correctement)
            print("\nConstruction programmatique des requêtes FOL...")
            queries_fol_obj = {}
            try:
                # Redéfinir Predicate/Constant avec les classes importées de commons.syntax
                Flies = Predicate("Flies", 1)
                Nests = Predicate("Nests_in_trees", 1)
                tina = Constant("tina")
                tweety = Constant("tweety")

                # Construire les FolAtom (le constructeur prend Predicate, List<Term>)
                # Créer une ArrayList Java pour les arguments
                args_tina = ArrayList([tina])
                args_tweety = ArrayList([tweety])

                # Utiliser JObject pour passer la liste Java de manière sûre
                queries_fol_obj["Flies(tina)"] = FolAtom(Flies, JObject(args_tina, "java.util.List"))
                queries_fol_obj["Flies(tweety)"] = FolAtom(Flies, JObject(args_tweety, "java.util.List"))
                queries_fol_obj["Nests_in_trees(tina)"] = FolAtom(Nests, JObject(args_tina, "java.util.List"))
                queries_fol_obj["Nests_in_trees(tweety)"] = FolAtom(Nests, JObject(args_tweety, "java.util.List"))

                print("✔️ Requêtes construites programmatiquement.")

            except Exception as e_build:
                print(f"❌ Erreur lors de la construction programmatique des requêtes: {e_build}")
                queries_fol_obj = None

            if queries_fol_obj:
                reasoner_delp = DelpReasoner(GeneralizedSpecificity())
                FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")

                print("\nÉvaluation des requêtes DeLP (construites programmatiquement):")
                for q_str, query_formula_obj in queries_fol_obj.items():
                    print(f"  Querying '{q_str}'...", end="")
                    try:
                        result_delp = reasoner_delp.query(delp_program, JObject(query_formula_obj, FolFormula_class))
                        print(f" Résultat: {result_delp}")
                    except jpype.JException as e_query_java:
                        print(f" ERREUR JAVA: {e_query_java.message()}")
                    except Exception as e_query_py:
                         print(f" ERREUR PYTHON: {e_query_py}")
            else:
                 print("\n❌ Construction des requêtes échouée, raisonnement sauté.")
        else:
            print("\n❌ Aucun programme DeLP chargé ou erreur de chargement, raisonnement sauté.")

    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour DeLP/Commons/FOL : {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()


--- 4.3 Defeasible Logic Programming (DeLP) ---
❌ ERREUR: JVM non démarrée.


### 4.4 Assumption-Based Argumentation (ABA)
<a id="4.4"></a>

*(Section à compléter avec l'exemple `AbaExample.java`)*

En ABA, certains littéraux sont désignés comme des **hypothèses** (assumptions). Les arguments sont dérivés en utilisant des règles logiques (similaires à ASPIC+) à partir de ces hypothèses. Une attaque d'un argument vers un autre se produit si la conclusion du premier est le **contraire** d'une hypothèse utilisée dans le second.

* **`AbaTheory`**: Contient les règles, l'ensemble des hypothèses, et la définition des contraires.
* **Logique sous-jacente**: Peut être PL ou FOL.
* **Raisonnement**: Souvent basé sur la conversion en AF de Dung (`FlatAbaReasoner`, `PreferredReasoner`).

In [ ]:
# --- 4.4 Assumption-Based Argumentation (ABA) ---
print("\n--- 4.4 Assumption-Based Argumentation (ABA) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ABA...")
    try:
        # Imports (inchangés)
        import jpype; from jpype.types import *; import pathlib; from java.io import StringReader
        from org.tweetyproject.arg.aba.parser import AbaParser
        from org.tweetyproject.arg.aba.syntax import AbaTheory, Assumption
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import Proposition, PlFormula
        from org.tweetyproject.logics.fol.parser import FolParser
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature
        from org.tweetyproject.logics.pl.sat import SatSolver, Sat4jSolver
        from org.tweetyproject.arg.aba.reasoner import FlatAbaReasoner, PreferredReasoner
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.arg.dung.syntax import DungTheory

        print("✔️ Imports ABA et dépendances réussis.")

        # --- Exemple 1: Propositional Logic (PL) ---
        print("\n--- Exemple ABA avec Logique Propositionnelle ---")
        # (Code PL inchangé, il fonctionnait)
        SatSolver.setDefaultSolver(Sat4jSolver())
        pl_parser_for_aba = PlParser()
        aba_parser_pl = AbaParser(pl_parser_for_aba)
        aba_pl_filename = "example2.aba"
        aba_pl_filepath = pathlib.Path("resources") / aba_pl_filename
        aba_theory_pl = None
        if not aba_pl_filepath.is_file():
            print(f"❌ ERREUR: Fichier ABA (PL) requis '{aba_pl_filepath}' non trouvé !")
        else:
            print(f"Chargement ABA (PL) depuis fichier: {aba_pl_filepath}")
            try:
                 aba_theory_pl = aba_parser_pl.parseBeliefBaseFromFile(str(aba_pl_filepath))
                 print("✔️ Théorie ABA (PL) chargée depuis fichier.")
            except Exception as e_fpl: print(f"❌ Erreur chargement fichier ABA (PL): {e_fpl}")

        if aba_theory_pl is not None:
            print("\nThéorie ABA (PL):", str(aba_theory_pl))
            reasoner_flat_pref = FlatAbaReasoner(Semantics.PREFERRED_SEMANTICS)
            reasoner_pref_aba = PreferredReasoner()
            assumption_pl = Assumption(Proposition("a"))
            print("\nRequêtes sur la théorie ABA (PL):")
            print(f" - Query '{assumption_pl}' (Flat Preferred)? {reasoner_flat_pref.query(aba_theory_pl, assumption_pl)}")
            print(f" - Query '{assumption_pl}' (ABA Preferred)? {reasoner_pref_aba.query(aba_theory_pl, assumption_pl)}")


        # --- Exemple 2: First-Order Logic (FOL) ---
        print("\n\n--- Exemple ABA avec Logique du Premier Ordre ---")
        aba_fol_filename = "smp_fol.aba"
        aba_fol_filepath = pathlib.Path("resources") / aba_fol_filename
        aba_theory_fol = None

        if not aba_fol_filepath.is_file():
             print(f"❌ ERREUR: Fichier ABA (FOL) requis '{aba_fol_filepath}' non trouvé !")
        else:
            fol_parser_for_aba = FolParser()
            # ** CORRECTION : Ajouter des sauts de ligne '\n' **
            sig_fol_str = """
            Male = {a, b}
            Female = {c, d}
            type(Pair(Male, Female))
            type(ContraryPair(Male, Female))
            type(MPrefers(Male, Female, Female))
            type(WPrefers(Female, Male, Male))
            """
            try:
                 sig_fol_aba = fol_parser_for_aba.parseSignature(sig_fol_str)
                 fol_parser_for_aba.setSignature(sig_fol_aba)
                 print("✔️ Signature FOL pour ABA définie.")

                 aba_parser_fol = AbaParser(fol_parser_for_aba)
                 # Important si les termes sont séparés par ';' dans le .aba
                 # smp_fol.aba utilise bien ';', donc on décommente :
                 aba_parser_fol.setSymbolComma(";")
                 print("ℹ️ Utilisation de ';' comme séparateur pour le parser ABA FOL.")


                 print(f"Chargement ABA (FOL) depuis fichier: {aba_fol_filepath}")
                 try:
                      aba_theory_fol = aba_parser_fol.parseBeliefBaseFromFile(str(aba_fol_filepath))
                      print("✔️ Théorie ABA (FOL) chargée depuis fichier.")
                 except Exception as e_ffol: print(f"❌ Erreur chargement fichier ABA (FOL): {e_ffol}")

                 if aba_theory_fol is not None:
                       print("\nThéorie ABA (FOL):\n", str(aba_theory_fol))
                       # Raisonnement ABA (FOL)
                       reasoner_flat_stable_fol = FlatAbaReasoner(Semantics.STABLE_SEMANTICS)
                       reasoner_pref_aba_fol = PreferredReasoner()
                       FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
                       # Vérifier que 'Pair(a,d)' est bien une assumption dans smp_fol.aba
                       # Elle l'est, comme 'Pair(b,c)', etc.
                       assumption_fol_query_str = "Pair(a,d)"
                       assumption_fol = Assumption(JObject(fol_parser_for_aba.parseFormula(assumption_fol_query_str), FolFormula_class))
                       print("\nRequêtes sur la théorie ABA (FOL):")
                       # print(f" - Modèles (Flat Stable): {reasoner_flat_stable_fol.getModels(aba_theory_fol)}")
                       print(f" - Query '{assumption_fol}' (ABA Preferred)? {reasoner_pref_aba_fol.query(aba_theory_fol, assumption_fol)}")

            except jpype.JException as e_sig_java: print(f"❌ Erreur Java lors de la config FOL pour ABA: {e_sig_java.message()}")
            except Exception as e_sig_py: print(f"❌ Erreur Python lors de la config FOL pour ABA: {e_sig_py}")

    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour ABA : {e}.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()


--- 4.4 Assumption-Based Argumentation (ABA) ---
❌ ERREUR: JVM non démarrée.


### 4.5 Argumentation Déductive (PL)
<a id="4.5"></a>

Cette approche, souvent associée à Besnard & Hunter, construit des arguments comme des paires `(Support, Conclusion)` où `Support` est un sous-ensemble minimal et consistant de la base de connaissances qui implique logiquement la `Conclusion`.

* **`DeductiveKnowledgeBase`**: Une simple `PlBeliefSet` dans l'implémentation de Tweety.
* **Argument**: Généré implicitement par le raisonneur.
* **Attaque**: Un argument `(S1, C1)` attaque `(S2, C2)` si `C1` est la négation d'un élément de `S2` (attaque "undercut" classique) ou la négation de `C2` (attaque "rebut" classique).
* **Raisonnement (`SimpleDeductiveReasoner`)**: Construit un arbre d'arguments/contre-arguments pour une requête donnée.
    * **`Categorizer`**: Détermine la force initiale d'un argument (ex: `ClassicalCategorizer`).
    * **`Accumulator`**: Agrège la force des arguments/contre-arguments dans l'arbre (ex: `SimpleAccumulator`).

In [ ]:
# --- 4.5 Argumentation Déductive (PL) ---
# Correction: Le package pour ClassicalCategorizer et SimpleAccumulator était 'categorizer' et 'accumulator', pas 'semantics'.
try:
    from org.tweetyproject.logics.pl.parser import PlParser
    from org.tweetyproject.arg.deductive.syntax import DeductiveKnowledgeBase
    from org.tweetyproject.arg.deductive.reasoner import SimpleDeductiveReasoner, AbstractDeductiveArgumentationReasoner
    # Correction des imports:
    from org.tweetyproject.arg.deductive.categorizer import ClassicalCategorizer
    from org.tweetyproject.arg.deductive.accumulator import SimpleAccumulator

    from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver # Souvent nécessaire en interne

    # --- Initialisation ---
    SatSolver.setDefaultSolver(Sat4jSolver())
    parser_ded = PlParser()
    kb_ded = DeductiveKnowledgeBase() # Est essentiellement un PlBeliefSet

    # Base de connaissances de l'exemple
    formulas_ded = ["s", "!s || h", "f", "!f || !h", "v", "!v || !h"]
    for f_str in formulas_ded: kb_ded.add(parser_ded.parseFormula(f_str))

    print("KB Déductive:\n", kb_ded)

    # --- Raisonnement ---
    # Utilise un catégoriseur classique et un accumulateur simple
    ded_reasoner = SimpleDeductiveReasoner(ClassicalCategorizer(), SimpleAccumulator())

    query_ded = parser_ded.parseFormula("h") # La requête est "h"
    result_ded = ded_reasoner.query(kb_ded, query_ded)

    print(f"\nQuery '{query_ded}'? {result_ded}") # Devrait être ACCEPTED ou REJECTED

except ImportError as e:
     print(f"❌ Erreur d'import pour Argumentation Déductive : {e}")
     print("   Vérifiez le JAR 'org.tweetyproject.arg.deductive'.")
except Exception as e:
     print(f"❌ Erreur lors de l'exécution de l'exemple Déductif: {e}")
     import traceback
     traceback.print_exc()

❌ Erreur d'import pour Argumentation Déductive : Attempt to create Java package 'org' without jvm
   Vérifiez le JAR 'org.tweetyproject.arg.deductive'.


### 4.6 Answer Set Programming (ASP)
<a id="4.6"></a>

ASP est un paradigme de programmation déclarative puissant pour résoudre des problèmes combinatoires complexes, souvent utilisé en représentation de connaissances et raisonnement. Un programme ASP est un ensemble de règles logiques (similaires à Prolog mais avec une sémantique différente basée sur les "modèles stables" ou "answer sets").

* **Syntaxe :** Règles de la forme `tete :- corps.` où `corps` est une conjonction de littéraux. Utilise la négation par défaut (`not`) et la négation classique (`-`). Permet les contraintes (règles sans tête) et les règles de choix/agrégats.
* **Sémantique :** Les "answer sets" sont des ensembles minimaux d'atomes vrais qui satisfont toutes les règles du programme.
* **Tweety :** Permet de définir des `Program`mes ASP (`org.tweetyproject.lp.asp.syntax`). Le raisonnement nécessite un **solveur externe** comme **Clingo** (de la suite Potassco). Tweety fournit `ClingoSolver` pour l'interfacer, ainsi qu'un `GringoGrounder` (Gringo est le "grounder" de Clingo qui instancie les variables).
* **Pré-requis :** Nécessite l'installation de **Clingo** ([Potassco](https://potassco.org/)) et la configuration du chemin vers l'exécutable Clingo dans `EXTERNAL_TOOLS['CLINGO']` (Cellule 9).

In [ ]:
# --- 4.6 Answer Set Programming (ASP) ---
print("\n--- 4.6 Answer Set Programming (ASP) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution de l'exemple ASP...")
    asp_imports_ok = False
    try:
        # Imports (inchangés)
        import jpype
        from jpype.types import *
        from java.util import ArrayList, List as JavaList, Collection
        from org.tweetyproject.lp.asp.syntax import (
            Program, ASPRule, ASPAtom, DefaultNegation, StrictNegation,
            ClassicalHead, AggregateHead, AggregateAtom, ASPHead, ASPBodyElement
        )
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate, Variable
        from org.tweetyproject.lp.asp.reasoner import ClingoSolver
        from org.tweetyproject.lp.asp.grounder import GringoGrounder
        from org.tweetyproject.lp.asp.semantics import AnswerSet

        # Récupérer get_tool_path
        if 'get_tool_path' not in globals() or 'EXTERNAL_TOOLS' not in globals():
            raise NameError("La fonction 'get_tool_path' ou 'EXTERNAL_TOOLS' n'est pas définie.")

        # Récupérer le chemin Clingo configuré par l'utilisateur
        CLINGO_PATH = get_tool_path('CLINGO') # Renvoie None si non configuré ou invalide

        print("✔️ Imports ASP réussis.")
        asp_imports_ok = True

        # --- Programme ASP Simple (Construction inchangée) ---
        print("\n--- Programme ASP Simple ---")
        p = ASPAtom("p"); r_atom = ASPAtom("r"); q = ASPAtom("q"); b_atom = ASPAtom("b") # Renommé r -> r_atom
        body1 = ArrayList(); body1.add(DefaultNegation(r_atom))
        r1_asp = ASPRule(ClassicalHead(p), JObject(body1, JavaList))
        body2 = ArrayList(); body2.add(StrictNegation(q)); body2.add(DefaultNegation(b_atom))
        r2_asp = ASPRule(ClassicalHead(r_atom), JObject(body2, JavaList))
        body3 = ArrayList(); body3.add(b_atom)
        r3_asp = ASPRule(ClassicalHead(StrictNegation(q)), JObject(body3, JavaList))
        body4 = ArrayList()
        r4_asp = ASPRule(ClassicalHead(b_atom), JObject(body4, JavaList))
        rules_prog1 = ArrayList([r1_asp, r2_asp, r3_asp, r4_asp])
        program1 = Program(JObject(rules_prog1, Collection))
        print("Programme 1:\n", str(program1))


        # --- Programme ASP Suspects (Construction inchangée) ---
        print("\n--- Programme ASP Suspects ---")
        motive = Predicate("motive", 1); guilty = Predicate("guilty", 1); innocent = Predicate("innocent", 1)
        harry = Constant("harry"); sally = Constant("sally")
        Suspect = Variable("Suspect")
        motive_h = ASPAtom(motive, [harry]); motive_s = ASPAtom(motive, [sally]) # Mettre les termes dans une liste
        guilty_h = ASPAtom(guilty, [harry])
        innocent_S = ASPAtom(innocent, [Suspect]); motive_S = ASPAtom(motive, [Suspect])
        guilty_S = ASPAtom(guilty, [Suspect])
        r1_sus = ASPRule(ClassicalHead(motive_h), JObject(ArrayList(), JavaList))
        r2_sus = ASPRule(ClassicalHead(motive_s), JObject(ArrayList(), JavaList))
        r3_sus = ASPRule(ClassicalHead(guilty_h), JObject(ArrayList(), JavaList))
        body4_sus = ArrayList(); body4_sus.add(motive_S); body4_sus.add(DefaultNegation(guilty_S))
        r4_sus = ASPRule(ClassicalHead(innocent_S), JObject(body4_sus, JavaList))
        rules_prog2 = ArrayList([r1_sus, r2_sus, r3_sus, r4_sus])
        program2 = Program(JObject(rules_prog2, Collection))
        print("Programme 2:\n", str(program2))


        # --- Raisonnement avec Clingo ---
        if not CLINGO_PATH:
            print("\n⚠️ Clingo non configuré ou chemin invalide dans EXTERNAL_TOOLS['CLINGO'] (Cellule 10).")
            print("   Veuillez installer Clingo (via pip/conda ou manuellement) et configurer le chemin.")
            print("   Raisonnement ASP sauté.")
        else:
            print(f"\nUtilisation de Clingo trouvé/configuré à: {CLINGO_PATH}")
            try:
                # Instancier AVEC le chemin
                solver_instance = ClingoSolver(JString(CLINGO_PATH))
                print(f"\nCalcul des Answer Sets pour Programme 2...")
                # getModels(Program)
                answer_sets_collection = solver_instance.getModels(program2)

                print(f"Answer Sets trouvés ({answer_sets_collection.size()}):")
                if answer_sets_collection.isEmpty():
                    print("   (Aucun Answer Set)")
                else:
                    count = 1
                    for ans_set in answer_sets_collection:
                        # Correction : ans_set est l'AnswerSet, on veut ses littéraux
                        atoms_in_set_obj = ans_set.getLiterals() # Renvoie Set<Literal>
                        # Convertir en liste de strings Python triée
                        atoms_in_set = ", ".join(sorted([str(atom) for atom in atoms_in_set_obj]))
                        print(f"   - AS {count}: {{{atoms_in_set}}}")
                        count += 1

                # --- Grounding (Optionnel) ---
                # (Le code de grounding reste identique mais indenté dans le 'else')
                print("\nTentative de Grounding...")
                try:
                    grounder_instance = GringoGrounder(JString(CLINGO_PATH))
                    ground_program = grounder_instance.getGroundProgram(program2)
                    print("   ✔️ Grounding réussi.")
                    print(f"     (Nombre de règles groundées: {ground_program.size()})")
                    # print("Programme Groundé:\n", str(ground_program)) # Potentiellement très long
                except Exception as e_ground:
                    print(f"   ❌ Erreur lors du grounding: {e_ground}")

            except jpype.JException as e_clingo_java:
                print(f"❌ Erreur Java lors de l'appel à Clingo: {e_clingo_java.message()}")
                print("   Vérifiez que le chemin vers Clingo est correct et que Clingo fonctionne.")
                # print(e_clingo_java.stacktrace())
            except Exception as e_clingo_py:
                print(f"❌ Erreur Python lors de l'appel à Clingo: {e_clingo_py}")
                import traceback; traceback.print_exc()


    # ... (Blocs except globaux inchangés) ...
    except ImportError as e: print(f"❌ Erreur d'import pour ASP : {e}. Vérifiez le JAR 'lp.asp'.")
    except jpype.JException as e_java: print(f"❌ Erreur Java générale ASP: {e_java.message()}")
    except Exception as e_gen: print(f"❌ Erreur Python inattendue ASP: {e_gen}"); import traceback; traceback.print_exc()


--- 4.6 Answer Set Programming (ASP) ---
❌ ERREUR: JVM non démarrée.


---

## Navigation

- **Precedent**: [Tweety-4-Belief-Revision.ipynb](Tweety-4-Belief-Revision.ipynb)
- **Suivant**: [Tweety-6-Advanced-Argumentation.ipynb](Tweety-6-Advanced-Argumentation.ipynb)
- **Index**: [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb)
